In [28]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Load the Data

In [29]:
# Read the CPI data
file_path = Path("./Resource/CPI_report_origin.xlsx")
df_CPI = pd.read_excel(file_path, header=11)

# Display sample data
df_CPI

c:\Users\KX\anaconda3\envs\dev\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual,HALF1,HALF2
0,1913,9.800,9.800,9.800,9.800,9.700,9.800,9.900,9.900,10.000,10.000,10.100,10.000,9.900,NaN,NaN
1,1914,10.000,9.900,9.900,9.800,9.900,9.900,10.000,10.200,10.200,10.100,10.200,10.100,10.000,NaN,NaN
2,1915,10.100,10.000,9.900,10.000,10.100,10.100,10.100,10.100,10.100,10.200,10.300,10.300,10.100,NaN,NaN
3,1916,10.400,10.400,10.500,10.600,10.700,10.800,10.800,10.900,11.100,11.300,11.500,11.600,10.900,NaN,NaN
4,1917,11.700,12.000,12.000,12.600,12.800,13.000,12.800,13.000,13.300,13.500,13.500,13.700,12.800,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2019,251.712,252.776,254.202,255.548,256.092,256.143,256.571,256.558,256.759,257.346,257.208,256.974,255.657,254.412,256.903
107,2020,257.971,258.678,258.115,256.389,256.394,257.797,259.101,259.918,260.280,260.388,260.229,260.474,258.811,257.557,260.065
108,2021,261.582,263.014,264.877,267.054,269.195,271.696,273.003,273.567,274.310,276.589,277.948,278.802,270.970,266.236,275.703
109,2022,281.148,283.716,287.504,289.109,292.296,296.311,296.276,296.171,296.808,298.012,297.711,296.797,292.655,288.347,296.963


## Prepare the Data

In [30]:
# Create dateframe without the annual and half data
df_CPI_no_annual_half = df_CPI.copy()
df_CPI_no_annual_half = df_CPI_no_annual_half.drop(['HALF1', 'HALF2', "Annual"], axis = 1)


In [31]:
# 
list_years = df_CPI_no_annual_half['Year']
list_months = df_CPI_no_annual_half.columns.tolist()[1:]
repeated_months = []

for year in range(len(list_years)):
    repeated_months = list_months + repeated_months

cpi_values = df_CPI_no_annual_half.iloc[:, 1:].values
cpi_values = cpi_values.reshape(-1)

In [32]:
# Reorganize the dataframe
history_df = pd.DataFrame({
    'year': df_CPI_no_annual_half['Year'].repeat(12),
    'month': repeated_months,
    'cpi_values': cpi_values
})
history_df.head()

,year,month,cpi_values
0,1913,Jan,9.8
0,1913,Feb,9.8
0,1913,Mar,9.8
0,1913,Apr,9.8
0,1913,May,9.7


In [33]:
month_name_to_number = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
}

# Map the 'month' column to month numbers
history_df['month'] = history_df['month'].map(month_name_to_number)

history_df.head()

,year,month,cpi_values
0,1913,1,9.8
0,1913,2,9.8
0,1913,3,9.8
0,1913,4,9.8
0,1913,5,9.7


In [34]:
history_df['year_month_decimal'] = history_df['year'] + history_df['month'] / 12
history_df.head()

,year,month,cpi_values,year_month_decimal
0,1913,1,9.8,1913.083333
0,1913,2,9.8,1913.166667
0,1913,3,9.8,1913.250000
0,1913,4,9.8,1913.333333
0,1913,5,9.7,1913.416667


In [35]:
# Get the year values as the feature data
X = history_df['year_month_decimal'].values.reshape(-1,1)

In [36]:
# Get the annual CPI value as target value
y = np.array([history_df['cpi_values']]).reshape(-1,1)

# Linear Regression

## Build the Linear Regression Model with Train data

In [37]:
# Create a model with scikit-learn
model = LinearRegression()

In [38]:
# Fit the data into the model
model.fit(X, y)

LinearRegression()

In [39]:
# Display the slope
print(f"Model's slope: {model.coef_[0][0]}")

Model's slope: 2.435661578176033


In [40]:
# Display the y-intercept
print(f"Model's y-intercept: {model.intercept_[0]}")

Model's y-intercept: -4707.443637373446


In [41]:
# Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_[0]} + {model.coef_[0][0]}X")

Model's formula: y = -4707.443637373446 + 2.435661578176033X


## Assess the Linear Regression Model

In [42]:
# Make predictions using the X set
predicted_y_values = model.predict(X)

In [43]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = model.score(X, y, sample_weight=None)
r2_LR = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2_LR}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.8372613023481741.
The r2 is 0.8372613023481741.
The mean squared error is 1183.9358589879046.
The root mean squared error is 34.40836902539707.
The standard deviation is 85.29403680892737.


## Predict the CPI for Future 80 Years

In [44]:
# Define X_future as the year values from 2024 to 2103
months_year = [int(month) for month in range(1, 13)]
future_years = np.arange(2024, 2104)
futureyear_repeated = [month for month in future_years for _ in range(12)]

futuremonth_repeated = []
for year in range(len(future_years)):
    futuremonth_repeated = futuremonth_repeated + months_year

future_dates = []
for month in range(len(futureyear_repeated)):
    date = futureyear_repeated[month] + futuremonth_repeated[month]/12
    future_dates.append(date)

future_dates = np.array(future_dates)
X_future = future_dates.reshape(-1,1)


In [45]:
# Predict the CPI values for future 80 years using the model
predicted_y = model.predict(X_future)

## Create Dataframe for Use

In [46]:
# Create the dataframe from 1913 to 2023
df_to_2023 = history_df[["year_month_decimal", "cpi_values"]]


In [47]:
# Create the dataframe from 1913 to 2023 with the predicted value
df_to_2023_predict = df_to_2023.copy()
df_to_2023_predict["Predicted CPI"] = predicted_y_values

In [48]:
# Create the dataframe from 2024 to 2103
df_to_2103 = pd.DataFrame(X_future, columns=["year_month_decimal"])
df_to_2103["cpi_values"] = predicted_y



In [49]:
# Create a dataframe that include the year and CPI value from 1913 to 2103
df_all_year = pd.concat([df_to_2023, df_to_2103])


## Visualize the Data

In [50]:
# Create a scatter plot of year versus the CPI value
scatter_plot_for_year_vs_CPI = df_to_2023.hvplot.scatter(
    x="year_month_decimal",
    xlabel="Year",
    y="cpi_values",
    ylabel="CPI Value",
    title="CPI Values Trend",
    color = "blue"
)
scatter_plot_for_year_vs_CPI

:Scatter   [year_month_decimal]   (cpi_values)

In [51]:
# Create a line plot to show the model
line_plot_model = df_to_2023_predict.hvplot.line(
    x="year_month_decimal",
    y="Predicted CPI",
    color="red"
)
line_plot_model

:Curve   [year_month_decimal]   (Predicted CPI)

In [52]:
# See the model predict with the origin data
scatter_plot_for_year_vs_CPI * line_plot_model

:Overlay
   .Scatter.I :Scatter   [year_month_decimal]   (cpi_values)
   .Curve.I   :Curve   [year_month_decimal]   (Predicted CPI)

In [53]:
# Create the line plot for future 80 years
line_plot_future = df_to_2103.hvplot.line(
    x="year_month_decimal",
    xlabel="Year",
    y="cpi_values",
    ylabel="CPI Values",
    color = "green"
) 
line_plot_future

:Curve   [year_month_decimal]   (cpi_values)

In [54]:
# Combine the plots
scatter_plot_for_year_vs_CPI * line_plot_model * line_plot_future

:Overlay
   .Scatter.I :Scatter   [year_month_decimal]   (cpi_values)
   .Curve.I   :Curve   [year_month_decimal]   (Predicted CPI)
   .Curve.II  :Curve   [year_month_decimal]   (cpi_values)